In [1]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession 
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import Row,SQLContext
import requests
import sys
import time
import json

In [2]:
ss = SparkSession.Builder() \
     .appName("Realtime") \
     .master("spark://spark-master-1:7077") \
     .config("spark.jars", "./spark-streaming-kafka-0-8-assembly_2.11-2.0.0-preview.jar") \
     .getOrCreate()
sc = ss.sparkContext
ssc = StreamingContext(sc, 5)
ssc.checkpoint("checkpoint_TwitterApp")
ss.sparkContext.setLogLevel('WARN')

In [3]:
brokers = 'kafka-1:9092,kafka-2:9092'
topic='tweet'

In [4]:
ks = KafkaUtils.createDirectStream(ssc, [topic], kafkaParams={"metadata.broker.list":brokers})

In [5]:
def process_rdd(time, rdd):
    try:
        df = rdd.take(10 + 1)
        top_tags  =[]
        tags_count=[]
        for h in df:
            top_tags.append(h[0])
            tags_count.append(h[1])
        send_df_to_dashboard(top_tags, tags_count)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [6]:
def send_df_to_dashboard(top_tags, tags_count):
    url = 'http://172.18.0.13:5001/updateData'
    request_data = {'label': str(top_tags), 'data': str(tags_count)}
    response = requests.post(url, data=request_data)

In [7]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

In [8]:
def filter(x):
    if x[0] == '#':
        if len(x) != 1:
            return True
    return False

In [9]:
lines = ks.map(lambda x: json.loads(x[1])['text'] )
hashtag = lines.flatMap(lambda line: line.split())\
                .filter(lambda w: filter(w))\
                .map(lambda x: (x, 1))\
                .updateStateByKey(aggregate_tags_count)\
                .transform(lambda rdd: rdd.sortBy(lambda x: x[1], ascending=False))
hashtag.foreachRDD(process_rdd)  
hashtag.pprint()
ssc.start()
time.sleep(60)
ssc.awaitTermination()

-------------------------------------------
Time: 2022-01-04 00:52:35
-------------------------------------------
('#Vaathi', 1)
('#SaiSoujanya', 1)
('#KaranKundrra', 1)
('#VarunSood', 1)
('#Commit', 1)
('#Together', 1)
('#BrunoLage', 1)
('#homemade', 1)
('#pallet', 1)
('#work', 1)
...

-------------------------------------------
Time: 2022-01-04 00:52:45
-------------------------------------------
('#Vaathi', 1)
('#SaiSoujanya', 1)
('#KaranKundrra', 1)
('#VarunSood', 1)
('#Commit', 1)
('#Together', 1)
('#BrunoLage', 1)
('#homemade', 1)
('#pallet', 1)
('#work', 1)
...

-------------------------------------------
Time: 2022-01-04 00:52:55
-------------------------------------------
('#Omicron_is_a_global_calamity', 3)
('#MUNWOL', 2)
('#Vaathi', 1)
('#SaiSoujanya', 1)
('#KaranKundrra', 1)
('#VarunSood', 1)
('#Commit', 1)
('#Together', 1)
('#BrunoLage', 1)
('#homemade', 1)
...

-------------------------------------------
Time: 2022-01-04 00:53:05
------------------------------------------

Py4JJavaError: An error occurred while calling o42.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/local/spark/python/pyspark/streaming/dstream.py", line 297, in <lambda>
    func = lambda t, rdd: oldfunc(rdd)
  File "<ipython-input-9-f156596fd993>", line 6, in <lambda>
    .transform(lambda rdd: rdd.sortBy(lambda x: x[1], ascending=False))
  File "/usr/local/spark/python/pyspark/rdd.py", line 699, in sortBy
    return self.keyBy(keyfunc).sortByKey(ascending, numPartitions).values()
  File "/usr/local/spark/python/pyspark/rdd.py", line 667, in sortByKey
    rddSize = self.count()
  File "/usr/local/spark/python/pyspark/rdd.py", line 1055, in count
    return self.mapPartitions(lambda i: [sum(1 for _ in i)]).sum()
  File "/usr/local/spark/python/pyspark/rdd.py", line 1046, in sum
    return self.m

-------------------------------------------
Time: 2022-01-04 00:53:35
-------------------------------------------
('#Omicron_is_a_global_calamity', 9)
('#Transportation', 3)
('#MUNWOL', 3)
('#realestate', 3)
('#IT', 2)
('#HAMSTER🔥', 2)
('#BTC', 2)
('#hamstercoin', 2)
('#metaverse', 2)
('#nft', 2)
...

-------------------------------------------
Time: 2022-01-04 00:53:45
-------------------------------------------
('#Omicron_is_a_global_calamity', 9)
('#MUNWOL', 4)
('#IT', 3)
('#Transportation', 3)
('#realestate', 3)
('#NFT', 2)
('#HAMSTER🔥', 2)
('#BTC', 2)
('#hamstercoin', 2)
('#metaverse', 2)
...

-------------------------------------------
Time: 2022-01-04 00:53:55
-------------------------------------------
('#Omicron_is_a_global_calamity', 9)
('#MUNWOL', 4)
('#IT', 3)
('#HAMSTER🔥', 3)
('#BTC', 3)
('#hamstercoin', 3)
('#metaverse', 3)
('#nft', 3)
('#Transportation', 3)
('#realestate', 3)
...

-------------------------------------------
Time: 2022-01-04 00:54:05
---------------------

-------------------------------------------
Time: 2022-01-04 00:57:55
-------------------------------------------
('#Omicron_is_a_global_calamity', 38)
('#BTC', 27)
('#metaverse', 19)
('#HAMSTER🔥', 18)
('#hamstercoin', 18)
('#nft', 18)
('#bitcoin', 18)
('#Gab', 12)
('#Transportation', 12)
('#NFT', 11)
...

-------------------------------------------
Time: 2022-01-04 00:58:05
-------------------------------------------
('#Omicron_is_a_global_calamity', 39)
('#BTC', 27)
('#metaverse', 19)
('#HAMSTER🔥', 18)
('#hamstercoin', 18)
('#nft', 18)
('#bitcoin', 18)
('#Transportation', 13)
('#Gab', 12)
('#NFT', 11)
...

-------------------------------------------
Time: 2022-01-04 00:58:15
-------------------------------------------
('#Omicron_is_a_global_calamity', 40)
('#BTC', 27)
('#metaverse', 19)
('#HAMSTER🔥', 18)
('#hamstercoin', 18)
('#nft', 18)
('#bitcoin', 18)
('#Gab', 13)
('#Transportation', 13)
('#NFT', 12)
...

-------------------------------------------
Time: 2022-01-04 00:58:25
------